In [3]:
from turtle import up
from pulp import *
import pandas as pd
import requests
import os
import numpy as np
from subprocess import Popen, DEVNULL
import itertools
import difflib

In [9]:
def get_pred_pts_data(gameweeks):
    fpl_name_dict = pd.read_csv(
        "../data/theFPLkiwi/ID_Dictionary.csv", encoding="cp1252"
    )[["Name", "FPL name"]]
    path = "../data/theFPLkiwi/FPL_projections_22_23/"
    files = [
        (int(f.strip("FPL_GW").strip(".csv")), os.path.join(path, f))
        for f in os.listdir(path)
        if os.path.isfile(os.path.join(path, f))
    ]
    files = [f for f in files if f[0] <= gameweeks[0]]
    index = np.argmax([f[0] for f in files])
    pred_pts_data = pd.read_csv(files[index][1])
    pred_pts_data = fpl_name_dict.merge(pred_pts_data, on="Name")
    empty_cols = ["GW xMins", "xPts if play", "Probability to play", "xPts", "npG"]
    pred_pts_data = pred_pts_data.drop(empty_cols, axis=1)
    week_cols = pred_pts_data.columns[9:]
    new_col_name = {
        col: f"{empty_cols[round(float(col)%1*10)]}_{round(float(col))}"
        for col in week_cols
    }
    pred_pts_data = pred_pts_data.rename(columns=new_col_name)
    pred_pts_data = pred_pts_data[
        ["FPL name", "Team", "Pos", "Price"] + [f"xPts_{w}" for w in gameweeks]
    ]
    return pred_pts_data

In [10]:
gameweeks = [8,9,10,11]
initial_squad = [543,380]
pred_pts_data = get_pred_pts_data(gameweeks)
pred_pts_data

,FPL name,Team,Pos,Price,xPts_8,xPts_9,xPts_10,xPts_11
0,Ramsdale,ARS,GK,4.9,3.798768,3.193286,2.904897,4.101898
1,Leno,FUL,GK,4.5,4.016978,3.536442,3.174960,4.358439
2,Turner,ARS,GK,3.9,0.095220,0.080855,0.074016,0.102788
3,Tierney,ARS,DEF,4.8,1.292469,1.219067,1.188628,3.263794
4,Gabriel,ARS,DEF,5.1,3.465034,3.221004,2.946787,3.570695
...,...,...,...,...,...,...,...,...
543,Gnonto,LEE,FWD,5.0,0.000000,0.380121,0.360071,0.365025
544,Aurier,NFO,DEF,3.0,0.100197,0.669839,0.824553,0.752234
545,Costa,WOL,FWD,5.5,1.714904,2.010478,1.393921,1.943720
546,Karius,NEW,GK,4.0,0.091430,0.049379,0.043224,0.033596


In [7]:
r = requests.get(
    f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{current_gw}/picks/"
)
picks_data = r.json()
initial_squad = [i["element"] for i in picks_data["picks"]]

FPL name    Surridge
Team             NFO
Pos              FWD
Price            4.8
xPts_8      0.675086
xPts_9      0.612378
xPts_10     0.633429
xPts_11     0.587766
Name: 380, dtype: object


KeyError: 123124